# LAB | Two Sample Hypothesis Tests

### Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [55]:
# import numpy and pandas

import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import ttest_ind, ttest_rel, ttest_1samp, chi2_contingency

## Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('../data/pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [11]:
leg = pokemon[pokemon['Legendary'] == True]
non_leg = pokemon[pokemon['Legendary'] == False]

print(f"Legendary Pokemons: {len(leg)}, Non-Legendary Pokemons: {len(non_leg)}")

Legendary Pokemons: 65, Non-Legendary Pokemons: 735


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [13]:
mean_leg = leg["Total"].mean()
std_leg = leg["Total"].std()
mean_non_leg = non_leg["Total"].mean()
std_non_leg = non_leg["Total"].std()
print(f"Legendary Pokemon - Mean: {mean_leg}, Std: {std_leg}")
print(f"Non-Legendary Pokemon - Mean: {mean_non_leg}, Std: {std_non_leg}")

Legendary Pokemon - Mean: 637.3846153846154, Std: 60.93738905315344
Non-Legendary Pokemon - Mean: 417.21360544217686, Std: 106.76041745713005


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [20]:
result = float(ttest_ind(leg["Total"], non_leg["Total"], equal_var=False).pvalue)
print(f"P-Value: {result}")

P-Value: 9.357954335957444e-47


What do you conclude from this test? Write your conclusions below.

In [22]:
# The P-Value is very small, so we can say that the difference is statistically significant.
# This suggests that Legendary Pokémon tend to have significantly different (and likely higher*) total stat scores compared to non-Legendary ones.
# 
# *definitely higher, they're Legendary Pokemon for a reason 😂

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [23]:
pokemon["Type 1"].value_counts()

Type 1
Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Rock         44
Electric     44
Ground       32
Ghost        32
Dragon       32
Dark         31
Poison       28
Fighting     27
Steel        27
Ice          24
Fairy        17
Flying        4
Name: count, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [27]:
# Vague instruction wording, I assume to use the 'Total' column for the comparison again

water = pokemon[pokemon['Type 1'] == 'Water']
non_water = pokemon[pokemon['Type 1'] != 'Water']

water_mean = water['Total'].mean()
non_water_mean = non_water['Total'].mean()
water_std = water['Total'].std()
non_water_std = non_water['Total'].std()

print(f"Mean comparison: {water_mean:.4f} vs {non_water_mean:.4f}")
print(f"Standard Deviation comparison: {water_std:.4f} vs {non_water_std:.4f}")

Mean comparison: 430.4554 vs 435.8590
Standard Deviation comparison: 113.1883 vs 121.0917


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [34]:
result2 = float(ttest_ind(water["Total"], non_water["Total"], equal_var=True).pvalue)
print(f"P-Value: {result2}")

P-Value: 0.6587140317488793


Write your conclusion below.

In [29]:
# The P-Value this time is very large (> 0.05) so we can say that the Total Stats between Water Pokemon and all other Pokemon together are not significantly different from each other.

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [40]:
result3 = float(ttest_rel(pokemon['Attack'], pokemon['Defense']).pvalue)
print(f"P-Value: {result3}")

P-Value: 1.7140303479358558e-05


Describe the results of the test in the cell below.

In [ ]:
# The P-Value is very small, indicating a statistically significant difference between Attack and Defense scores.
# Therefore, we reject the null hypothesis that the means of Attack and Defense are equal.
# This suggests that Pokémon tend to have either higher Attack or higher Defense, on average.

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [42]:
result4 = float(ttest_rel(pokemon['Sp. Atk'], pokemon['Sp. Def']).pvalue)
print(f"P-Value: {result4}")

P-Value: 0.3933685997548122


Describe the results of the test in the cell below.

In [43]:
# The P-Value here is high again, which indicates that these two Stats are not significantly different from each other.
# Therefore, we don't reject the hypothesis that the means of Sp. Atk and Sp. Def are the same.
# This suggests that Pokémon tend to have similar stats in terms of their special attack and special defense.

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [51]:
diff = pokemon['Defense'] - pokemon['Attack']

result5 = ttest_1samp(diff, popmean=0).pvalue
print(f"P-value: {result5}")

P-value: 1.7140303479358558e-05


In [52]:
# This one-sample t-test compares the difference between each Pokémon's Defense and Attack stats to 0.
# Since this is equivalent to a paired (matched pairs) t-test, the p-value matches the result from ttest_rel.
# This confirms that both tests support the same conclusion: there is a significant difference between Attack and Defense.

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [54]:
pokemon["Is_Water_Type"] = pokemon["Type 1"] == "Water"

contingency_table = pd.crosstab(pokemon["Legendary"], pokemon["Is_Water_Type"])

contingency_table

Is_Water_Type,False,True
Legendary,,
False,627,108
True,61,4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [58]:
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-squared statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)

Chi-squared statistic: 2.9429200762850503
P-value: 0.0862546724955095
Degrees of freedom: 1
Expected frequencies:
[[632.1 102.9]
 [ 55.9   9.1]]


Based on a 95% confidence, should we reject the null hypothesis?

In [59]:
# The p-value from the chi-squared test is 0.086, which is greater than the significance level of 0.05.
# Therefore, we fail to reject the null hypothesis.
# This means there is not enough evidence to conclude that Legendary status and Water type are significantly associated.